<a href="https://colab.research.google.com/github/Maxxx-VS/Test_task/blob/master/Sushi_test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тестовое задание

Постановка задачи:
В оффлайн-продажах важно оценивать предполагаемый поток клиентов и объем проданной продукции. В рамках данного тестового задания необходимо выполнить следующие шаги:
1.	На основании предоставленных исходных данных выберите и сформируйте список уникальных номенклатурных позиций, которые присутствуют в продажах.
2.	Подготовьте «силуэт продаж» – выявите и опишите характерный профиль продаж по каждой номенклатурной позиции, учитывая сезонность по дням недели и месяцам. Силуэт должен отражать типичные изменения объема продаж во времени.
3.	Используя сформированный силуэт продаж, подготовьте предварительный прогноз объема продаж на завтрашний день для каждой номенклатурной позиции (завтрашним считается день, следующий за днем сдачи тестового задания).
4.	С помощью инструментов машинного обучения проведите пост-обработку и корректировку предварительного прогноза, полученного по силуэту продаж, до более точного значения.
Предоставьте итоговый результат в виде таблицы с перечнем уникальных номенклатурных позиций и соответствующим прогнозом продаж. Дополнительно кратко опишите используемый подход и методику расчета и корректировки прогноза.
Из 3 представленных на выбор файлов (данные за прошлый год по трем разным торговым точкам), выбрать нужно только один (в решении можно использовать все три).

Описание переменных:
CloseTime – время чека
DishAmountInt – количество позиций (целевая переменная)
DishDiscountSumInt – сумма в рублях
DishName – название блюда
OpenDate.Typed– дата заказа
RestorauntGroup– название торговой точки

Рекомендации:
Цель данного тестового посмотреть на ход решения и рассуждений.
Укажите, какие переменные используются для построения силуэта продаж и модели, почему именно они, какие подходы используются в решении (приветствуется добавление собственных признаков). Никаких ограничений в плане выбора технологий прогнозирования нет (кроме, конечно, здравого смысла). Как итог нужно презентовать код решения с пояснениями (возможно, ноутбук в google colab или jupyter) и ответить на возникающие вопросы. Готовить слайды рр на данном этане не нужно.


## **0.** РАБОТА С ДАННЫМИ

> Развернуть



In [1]:
# Загрузка данных, сохраненных в Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('''Необходимо указать  № торговой точки:

        №1 - Дианова, 14
        №2 - Энтузиастов, 2/1
        №3 - Патриса Лумумбы, 4б''', '\n')
choice = int(input("Выбор торговой точки: "))

Mounted at /content/drive
Необходимо указать  № торговой точки:

        №1 - Дианова, 14
        №2 - Энтузиастов, 2/1
        №3 - Патриса Лумумбы, 4б 

Выбор торговой точки: 1


In [2]:
# Формируем путь к Excel файлу
address_list = ['id_точки 16.xlsx', 'id_точки 34.xlsx', 'id_точки 597.xlsx']
path = '/content/drive/MyDrive/Sushi_test_task/' + address_list[choice-1]
print(path)

/content/drive/MyDrive/Sushi_test_task/id_точки 16.xlsx


In [3]:
# Создаем датафрейм
import pandas as pd
df = pd.read_excel(path)

In [4]:
# Вывод структуры датафрейма
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326619 entries, 0 to 326618
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CloseTime           326619 non-null  datetime64[ns]
 1   DishAmountInt       326619 non-null  int64         
 2   DishDiscountSumInt  326619 non-null  int64         
 3   DishName            326619 non-null  object        
 4   RestorauntGroup     326619 non-null  object        
 5   OpenDate.Typed      326619 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 15.0+ MB


In [5]:
# Изучим статистические показатели датафрейма
# count: количество непустых значений, mean: среднее значение, std: стандартное отклонение, min: минимальное значение, 25%, 50%, 75%: квартили, max: максимальное значение
df.describe()

,CloseTime,DishAmountInt,DishDiscountSumInt,OpenDate.Typed
count,326619,326619.000000,326619.000000,326619
mean,2024-08-14 22:06:43.376683776,2.549610,160.189435,2024-08-14 05:28:06.996776448
min,2024-02-01 10:03:47,0.000000,0.000000,2024-02-01 00:00:00
25%,2024-05-10 17:31:41,1.000000,13.000000,2024-05-10 00:00:00
50%,2024-08-16 15:27:35,1.000000,82.000000,2024-08-16 00:00:00
75%,2024-11-15 16:27:12,3.000000,190.000000,2024-11-15 00:00:00
max,2025-02-28 21:17:26,60.000000,6500.000000,2025-02-28 00:00:00
std,NaN,2.699514,240.291049,NaN


In [6]:
# Вывод размерности датафрейма
df.shape

(326619, 6)

In [7]:
# Вывод наименования столбцов датафрейма
df.columns

Index(['CloseTime', 'DishAmountInt', 'DishDiscountSumInt', 'DishName',
       'RestorauntGroup', 'OpenDate.Typed'],
      dtype='object')

In [8]:
# Переименуем столбец для исключения точки в названии
df = df.rename(columns={'OpenDate.Typed': 'OpenDate_Typed'})

In [9]:
# УДАЛЯЕМ СТРОКИ С ПУСТЫМИ ЗНАЧЕНИЯМИ В ДАТАФРЕЙМЕ И СО ЗНАЧЕНИЯМИ NaN
import numpy as np
df = df.replace('', np.nan).dropna()

# УДАЛЯЕМ СТРОКИ СО НУЛЕВЫМИ ЗНАЧЕНИЯМИ В ДАТАФРЕЙМЕ
collumns_to_check = ['DishAmountInt', 'DishDiscountSumInt']
df = df[(df[collumns_to_check] >= 1).all(axis=1)]

In [10]:
# Вывод структуры датафрейма после очистки от неинформативных значений
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 320650 entries, 0 to 326618
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CloseTime           320650 non-null  datetime64[ns]
 1   DishAmountInt       320650 non-null  int64         
 2   DishDiscountSumInt  320650 non-null  int64         
 3   DishName            320650 non-null  object        
 4   RestorauntGroup     320650 non-null  object        
 5   OpenDate_Typed      320650 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 17.1+ MB


In [11]:
# Учитывая, что данные относятся к одной торговой точке - удаляем признак "RestorauntGroup"
df = df.drop("RestorauntGroup", axis=1)
# Удаляем дублирующий признак "CloseTime"
df = df.drop("CloseTime", axis=1)

In [12]:
df

,DishAmountInt,DishDiscountSumInt,DishName,OpenDate_Typed
0,6,228,Лава МСК рол,2025-02-08
1,1,5,ПАКЕТ МАЙКА ОДНОР,2025-02-08
2,1,25,Контейнер большой одн,2025-02-08
3,2,20,ВАСАБИ ЗАВОДСКАЯ ФАСОВКА,2025-02-08
4,3,36,СОУС СОЕВЫЙ ЗАВОДСКАЯ ФАСОВКА,2025-02-08
...,...,...,...,...
326614,2,20,ВАСАБИ ЗАВОДСКАЯ ФАСОВКА,2025-02-28
326615,5,125,Калифорния акция МСК 10ролл,2025-02-28
326616,5,235,Филадельфия МСК ролл,2025-02-28
326617,1,5,ПАКЕТ МАЙКА ОДНОР,2025-02-28


In [13]:
# Объединяем данные с одинаковыми признаками 'OpenDate_Typed', 'DishName' и складываем 'DishAmountInt' и 'DishDiscountSumInt'
df = df.groupby(['DishName', 'OpenDate_Typed']).agg({
    'DishAmountInt': 'sum',
    'DishDiscountSumInt': 'sum'}).reset_index()

In [14]:
# Меняем тип данных признака 'CloseTime'
df['OpenDate_Typed'] = pd.to_datetime(df['OpenDate_Typed'])

# Извлекаем компоненты и раскладываем на год/месяц/день
df['Year'] = df['OpenDate_Typed'].dt.year
df['Month'] = df['OpenDate_Typed'].dt.month
df['Week'] = df['OpenDate_Typed'].dt.isocalendar().week
df['Weekday'] = df['OpenDate_Typed'].dt.dayofweek

In [15]:
# Проверка на соответсвие всех целочисленных значений в столбцах 'DishAmountInt', 'DishDiscountSumInt'
def check_column_type_int(df, column_name, expected_type):
    return all(isinstance(item, expected_type) for item in df[column_name])

print(check_column_type_int(df, 'DishAmountInt', int))  # True
print(check_column_type_int(df, 'DishDiscountSumInt', int))  # True
print(check_column_type_int(df, 'DishName', str))  # True


# Проверка на соответсвие всех datetime значений в столбце 'OpenDate_Typed'
def is_datetime_column(df, column_name):
    return pd.api.types.is_datetime64_ns_dtype(df[column_name])

print(is_datetime_column(df, 'OpenDate_Typed'))  # True

True
True
True
True


In [19]:
df

,DishName,DishAmountInt,DishDiscountSumInt,Year,Month,Week,Weekday
0,&Набор Калифорния подаро 8рол,3,3,2024,2,5,3
1,&Набор Калифорния подаро 8рол,1,1,2024,2,5,4
2,&Набор Калифорния подаро 8рол,3,3,2024,2,5,5
3,&Набор Калифорния подаро 8рол,5,5,2024,2,5,6
4,&Набор Калифорния подаро 8рол,2,2,2024,2,6,0
...,...,...,...,...,...,...,...
24485,Ясай 10рол,17,374,2025,2,9,0
24486,Ясай 10рол,29,638,2025,2,9,1
24487,Ясай 10рол,18,396,2025,2,9,2
24488,Ясай 10рол,6,132,2025,2,9,3


## **1.** СПИСОК УНИКАЛЬНЫХ НОМЕНКЛАТУРНЫХ ПОЗИЦИЙ

> Развернуть

In [25]:
# Обращаемся к столбцу и и применяем метод .unique()
unique_values = df['DishName'].unique()
print(f'Общее количество уникальных номенклатурных позиций = {len(unique_values)} ед.', '\n')
unique_values

Общее количество уникальных номенклатурных позиций = 178 ед. 



array(['&Набор Калифорния подаро 8рол', '&Набор Калифорния подарок 8рол',
       '&Набор Популярный Дуэт 8рол', '&Набор Приведи друга Бостон 8рол',
       '&Онигири креветка подарок', 'Rollbox Аппетитный 10шт',
       'Rollbox Любимый 10шт', 'Rollbox Популярный 10шт',
       'Rollbox Суши Бокс 5шт', 'Rollbox Филадельфия 10шт',
       'Rollbox Фитнес-Бокс 13шт', 'Rollbox Фиш микс 10шт',
       'Амазонка запеч 10рол', 'Амазонка запеч МСК',
       'Амазонка запеч ролл10', 'Аризона 10 рол', 'Аризона 10рол',
       'Аризона МСК ролл', 'Бансай 10рол', 'Бансай МСК ролл',
       'Бонито 10рол', 'Бонито МСК ролл', 'Бостон 10рол',
       'Бостон МСК ролл', 'Босфор запеч 10рол', 'Босфор запеч МСК',
       'ВАСАБИ ЗАВОДСКАЯ ФАСОВКА', 'ВИЛОЧКА В УПАКОВКЕ ОДНОР',
       'ВИЛОЧКА ОДНОР', 'ВОДА Б/Г 0,33л', 'ВОДА Б/Г СУШИ - МАРКЕТ 0,33л',
       'ВОДА В АССОРТ 0,5 л', 'Валенсия МСК ролл',
       'Вок с беконом в сливочном соусе 330гр',
       'Вок с беконом в терияки соусе 330гр',
       'Вок с креветк

## **2.** СИЛУЭТ ПРОДАЖ

> Развернуть

In [ ]:
# Графически изображаем силуэт продаж с отбором по временным критериям
import matplotlib.pyplot as plt

